# dictionary와 tuple보다는 핼퍼 클래스로 관리하자

Python에 내장되어 있는 dictionary 타입은 객체의 수명이 지속되는 동안 동적인 내부 상태를 관리하는 용도로 좋다.  
__'동적'이란 예상하지 못한 식별자들을 관리해야 하는 상황을 뜻한다.__

이름을 모르는 학생 집단의 성적을 기록하는 클래스

In [1]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = []
        
    def report_grade(self, name, score):
        self._grades[name].append(score)
        
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

클래스의 사용법은 간단하다.

In [3]:
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
# ...
print(book.average_grade('Isaac Newton'))

90.0


dictionary는 정말 사용하기 쉬워서 과도하게 쓰다가 코드를 취약하게 작성할 위험이 있다.

SimpleGradebook 클래스를 확장해서 성적을 과목별로 저장하는 예시

In [10]:
class BySubjectGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append(grade)
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grdes in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

report_grade와 average_grade method는 여러 단계의 dictionary를 처리한다.

In [11]:
book = BySubjectGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 75)
book.report_grade('Albert Einstein', 'Math', 65)
book.report_grade('Albert Einstein', 'Gym', 90)
book.report_grade('Albert Einstein', 'Gym', 95)

이때 요구사항이 변해서 점수가 차지하는 비중을 달리하도록 해보자.  
이를 위해 가장 안쪽 dictionary를 변경해서 키를 값에 매핑하지 않고, 성적과 비중을 담은 tuple에 mapping한다.

In [12]:
class WeightedGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append((score, weight))
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                # ...
                total_weight += sum(score * weight)
                
        return total / count

클래스를 사용하는 방법이 어려워졌고 인수에 있는 숫자들의 의미가 불명확하다.  
계층이 한 단계가 넘는 중첩은 피해야 한다. 여러 계층으로 중첩하면 다른 프로그래머들이 코드를 이해하기 어려워지고 유지보수의 악몽에 빠지게 된다.

관리하기 복잡하다고 느끼는 즉시 클래스로 옮겨야 한다.

## 클래스 리팩토링

의존 관계에서 가장 아래에 있는 성적부터 클래스로 옮겨본다. 간단한 정보를 담기에 클래스는 너무 무거워 보인다. 성적은 변하지 않으니 tuple을 사용하는 게 더 적절해 보인다.

In [13]:
grades = []
grades.append((95, 0.45))
# ...
total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grade = total / total_weight

일반 tuple은 위치에 의존한다. 더 많은 정보를 연관지으려면 이제 tuple을 사용하는 곳을 모두 찾아 아이템을 세 개로 수정해야 한다.

In [14]:
grades = []
grades.append((95, 0.45, 'Great job'))
total = sum(score * weight for score, weight, _ in grades)
total_weight = sum(weight for _, weight, _ in grades)
average_grade = total / total_weight

tuple을 점점 더 길게 확장하는 패턴은 dictionary의 계층을 깊게 도는 방식과 유사하다. 따라서 다른 방법을 고려해야한다.

collections 모듈의 namedtuple 타입이 정확히 이런 요구에 부합한다. nmaedtuple을 이용하면 작은 immutable data class를 쉽게 정의할 수 있다.

In [15]:
import collections
Grade = collections.namedtuple('Grade', ('score', 'weight'))

불변 데이터 클래스는 위치 인수나 키워드 인수로 생성할 수 있다. 필드는 이름이 붙은 속성으로 접근할 수 있다. 이름이 붙은 속성이 있으면 나중에 요구 사항이 또 변해서 단순 데이터 컨테이너에 동작을 추가해야 할 때 namedtuple에서 직접 작성한 클래스로 쉽게 바꿀 수 있다.

In [25]:
class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [26]:
class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [27]:
class Gradebook(object):
    def __init__(self):
        self._students = {}
    
    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]

In [28]:
book = Gradebook()
albert = book.student('Albert Einstein')
math = albert.subject('Math')
math.report_grade(80, 0.10)
# ...
print(albert.average_grade())

80.0


필요하면 이전 형태의 API 스타일로 작성한 코드를 새로 만든 객체 계층 스타일로 바꿔주는 하위 호환용 메서드를 작성해도 된다.